User Database - Simple Test
===========================

In [1]:
from sqlalchemy import Table, Column, Integer, String, DateTime, MetaData, join, ForeignKey, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import column_property, sessionmaker

In [2]:
metadata = MetaData()

In [3]:
user_table = Table('user', metadata,
                  Column('id', Integer, primary_key=True),
                  Column('name', String)
            )
address_table = Table('address', metadata,
                Column('id', Integer, primary_key = True),
                Column('user_id', Integer, ForeignKey('user.id')),
                Column('email_address', String)
            )

In [4]:
user_address_join = join(user_table, address_table)

In [5]:
engine = create_engine('sqlite:///users.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

In [6]:
class AddressUser(Base):
    __table__ = user_address_join
    
    id = column_property(user_table.c.id, address_table.c.user_id)
    address_id = address_table.c.id
    
    def __repr__(self):
        return '< AddressUser [{self.name} | {self.email_address}] >'.format(self=self)

In [7]:
Base.metadata.create_all(engine)
metadata.create_all(engine)

2016-11-12 21:58:06,082 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-11-12 21:58:06,092 INFO sqlalchemy.engine.base.Engine ()
2016-11-12 21:58:06,109 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-11-12 21:58:06,157 INFO sqlalchemy.engine.base.Engine ()
2016-11-12 21:58:06,164 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("user")
2016-11-12 21:58:06,174 INFO sqlalchemy.engine.base.Engine ()
2016-11-12 21:58:06,187 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("address")
2016-11-12 21:58:06,213 INFO sqlalchemy.engine.base.Engine ()
2016-11-12 21:58:06,223 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE user (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (id)
)


2016-11-12 21:58:06,301 INFO sqlalchemy.engine.base.Engine ()
2016-11-12 21:58:06,552 INFO sqlalchemy.engine.base.Engine COMMIT
2016-11-12 21:58:06,561 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE

In [8]:
a = AddressUser()
a.name = 'John Doe'
a.email_address = 'email@address.com'

In [9]:
b = AddressUser(name='Jane Doe', email_address = 'address@email.com')

In [10]:
session = Session()
session.add_all([a, b])
session.commit()
session.close()

2016-11-12 21:58:18,368 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-11-12 21:58:18,388 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name) VALUES (?)
2016-11-12 21:58:18,467 INFO sqlalchemy.engine.base.Engine ('John Doe',)
2016-11-12 21:58:18,471 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name) VALUES (?)
2016-11-12 21:58:18,476 INFO sqlalchemy.engine.base.Engine ('Jane Doe',)
2016-11-12 21:58:18,480 INFO sqlalchemy.engine.base.Engine INSERT INTO address (user_id, email_address) VALUES (?, ?)
2016-11-12 21:58:18,592 INFO sqlalchemy.engine.base.Engine (1, 'email@address.com')
2016-11-12 21:58:18,599 INFO sqlalchemy.engine.base.Engine INSERT INTO address (user_id, email_address) VALUES (?, ?)
2016-11-12 21:58:18,608 INFO sqlalchemy.engine.base.Engine (2, 'address@email.com')
2016-11-12 21:58:18,617 INFO sqlalchemy.engine.base.Engine COMMIT


In [11]:
class User(Base):
    __table__ = user_table
    
    def __repr__(self):
        return '< User [{}] >'.format(self.name)
    
class Address(Base):
    __table__ = address_table
    
    def __repr__(self):
        return '< Address [{}] >'.format(self.email_address)

In [12]:
session = Session()

In [13]:
session.query(User).all()

2016-11-12 21:58:31,391 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-11-12 21:58:31,398 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name 
FROM user
2016-11-12 21:58:31,430 INFO sqlalchemy.engine.base.Engine ()


[< User [John Doe] >, < User [Jane Doe] >]

In [14]:
session.query(Address).all()

2016-11-12 21:58:34,691 INFO sqlalchemy.engine.base.Engine SELECT address.id AS address_id, address.user_id AS address_user_id, address.email_address AS address_email_address 
FROM address
2016-11-12 21:58:34,703 INFO sqlalchemy.engine.base.Engine ()


[< Address [email@address.com] >, < Address [address@email.com] >]

In [15]:
session.query(AddressUser).all()

2016-11-12 21:58:37,732 INFO sqlalchemy.engine.base.Engine SELECT address.id AS address_id, user.id AS user_id, address.user_id AS address_user_id, user.name AS user_name, address.email_address AS address_email_address 
FROM user JOIN address ON user.id = address.user_id
2016-11-12 21:58:37,815 INFO sqlalchemy.engine.base.Engine ()


[< AddressUser [John Doe | email@address.com] >,
 < AddressUser [Jane Doe | address@email.com] >]

In [16]:
session.close()

2016-11-12 21:58:40,928 INFO sqlalchemy.engine.base.Engine ROLLBACK


Letter Database - Complicated Case
==================================

In [1]:
from sqlalchemy import Table, Column, DateTime, Integer, String, MetaData, join, ForeignKey, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import column_property, sessionmaker, relationship
from datetime import datetime
from IPython.display import HTML

In [2]:
engine = create_engine('sqlite:///letters.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

In [3]:
class Department(Base):
    __tablename__ = 'departments'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    def __init__(self, name, users=None):
        self.name = name
        self.users = users or []
        
    def __repr__(self):
        return '<Department {}>'.format(self.name)

class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    username = Column(String, unique=True)
    email = Column(String)
    department_id = Column(Integer, ForeignKey('departments.id'))
    department = relationship('Department', backref='users')
    
    def __init__(self, username, email, department):
        self.username, self.email, self.department = username, email, department
        
    def __repr__(self):
        return '<User {s.username} | {s.email} | {s.department}>'.format(s=self)
    
class Letter(Base):
    __tablename__ = 'letter'
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    description = Column(String)
    from_id = Column(Integer, ForeignKey('users.id'))
    from_user = relationship('User', foreign_keys=[from_id])
    to_id = Column(Integer, ForeignKey('users.id'))
    to_user = relationship('User', foreign_keys=[to_id])
    datetime = Column(DateTime)
    
    def __init__(self, title, description, from_user, to_user, datetime):
        self.title, self.description = title, description
        self.from_user, self.to_user = from_user, to_user
        self.datetime = datetime
        
    def __repr__(self):
        return '''[ Letter: {s.title} 
From: {f.username} ({f.email})
To: {t.username} ({t.email})
Time: {s.datetime}

{s.description}]'''.format(s=self, f=self.from_user, t=self.to_user)

In [4]:
user_table, letter_table, dep_table = User.__table__, Letter.__table__, Department.__table__
from_user_table = user_table.alias('from_user')
to_user_table = user_table.alias('to_user')
from_join = join(letter_table, from_user_table, from_user_table.c.id == letter_table.c.from_id)
letter_join = join(from_join, to_user_table, to_user_table.c.id == from_join.c.letter_from_id)

In [5]:
for col_name, col in letter_join.c.items():
    col.key = col_name

letter_join.c.items()

[('letter_id',
  Column('id', Integer(), table=<letter>, key='letter_id', primary_key=True, nullable=False)),
 ('letter_title',
  Column('title', String(), table=<letter>, key='letter_title')),
 ('letter_description',
  Column('description', String(), table=<letter>, key='letter_description')),
 ('letter_from_id',
  Column('from_id', Integer(), ForeignKey('users.id'), table=<letter>, key='letter_from_id')),
 ('letter_to_id',
  Column('to_id', Integer(), ForeignKey('users.id'), table=<letter>, key='letter_to_id')),
 ('letter_datetime',
  Column('datetime', DateTime(), table=<letter>, key='letter_datetime')),
 ('from_user_id',
  Column('id', Integer(), table=<from_user>, key='from_user_id', primary_key=True, nullable=False)),
 ('from_user_username',
  Column('username', String(), table=<from_user>, key='from_user_username')),
 ('from_user_email',
  Column('email', String(), table=<from_user>, key='from_user_email')),
 ('from_user_department_id',
  Column('department_id', Integer(), Forei

In [6]:
class LetterDetail(Base):
    __table__ = letter_join
    __table__.columns = letter_join.columns
    
    id = column_property(letter_table.c.id)
    from_department = User.department
    to_department = User.department
    
    def _repr_html_(self):
        headers = '<tr><th>{}</th></tr>'.format('</th><th>'.join([i[0] for i in self.__table__.columns]))
        details = '<tr><td>{}</td></tr>'.format('</td><td>'.join(getattr(self, i[0]) for i in self.__table__.columns))
        return '<table>{}{}</table>'.format(headers, details)

In [7]:
Base.metadata.create_all(engine)

2016-11-12 22:00:10,701 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-11-12 22:00:10,710 INFO sqlalchemy.engine.base.Engine ()
2016-11-12 22:00:10,723 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-11-12 22:00:10,743 INFO sqlalchemy.engine.base.Engine ()
2016-11-12 22:00:10,756 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("letter")
2016-11-12 22:00:10,770 INFO sqlalchemy.engine.base.Engine ()
2016-11-12 22:00:10,809 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2016-11-12 22:00:10,815 INFO sqlalchemy.engine.base.Engine ()
2016-11-12 22:00:10,819 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("departments")
2016-11-12 22:00:10,828 INFO sqlalchemy.engine.base.Engine ()
2016-11-12 22:00:10,842 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE departments (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (id)
)


2016-11-12 22:00:10,867 INFO sqlalchemy

In [8]:
d1 = Department('Customer Relations')
d2 = Department('Human Resources')
john = User('john_doe', 'john@email.com', d1)
jane = User('jane_doe', 'jane@email.com', d2)
letter = Letter('Hello World', 'This is a test letter.\nHELLO WORLD!\n:)', john, jane, datetime.now())

In [9]:
session = Session()
session.add_all([d1, d2, john, jane, letter])
session.commit()
session.close()

2016-11-12 22:00:14,608 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-11-12 22:00:14,614 INFO sqlalchemy.engine.base.Engine INSERT INTO departments (name) VALUES (?)
2016-11-12 22:00:14,622 INFO sqlalchemy.engine.base.Engine ('Customer Relations',)
2016-11-12 22:00:14,637 INFO sqlalchemy.engine.base.Engine INSERT INTO departments (name) VALUES (?)
2016-11-12 22:00:14,644 INFO sqlalchemy.engine.base.Engine ('Human Resources',)
2016-11-12 22:00:14,648 INFO sqlalchemy.engine.base.Engine INSERT INTO users (username, email, department_id) VALUES (?, ?, ?)
2016-11-12 22:00:14,653 INFO sqlalchemy.engine.base.Engine ('john_doe', 'john@email.com', 1)
2016-11-12 22:00:14,661 INFO sqlalchemy.engine.base.Engine INSERT INTO users (username, email, department_id) VALUES (?, ?, ?)
2016-11-12 22:00:14,670 INFO sqlalchemy.engine.base.Engine ('jane_doe', 'jane@email.com', 2)
2016-11-12 22:00:14,690 INFO sqlalchemy.engine.base.Engine INSERT INTO letter (title, description, from_id, to_id, date

In [10]:
session = Session()

In [11]:
session.query(Department).all()

2016-11-12 22:00:17,584 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-11-12 22:00:17,589 INFO sqlalchemy.engine.base.Engine SELECT departments.id AS departments_id, departments.name AS departments_name 
FROM departments
2016-11-12 22:00:17,592 INFO sqlalchemy.engine.base.Engine ()


[<Department Customer Relations>, <Department Human Resources>]

In [12]:
session.query(User).all()

2016-11-12 22:00:19,855 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.department_id AS users_department_id 
FROM users
2016-11-12 22:00:19,864 INFO sqlalchemy.engine.base.Engine ()


[<User john_doe | john@email.com | <Department Customer Relations>>,
 <User jane_doe | jane@email.com | <Department Human Resources>>]

In [13]:
session.query(Letter).all()

2016-11-12 22:00:21,892 INFO sqlalchemy.engine.base.Engine SELECT letter.id AS letter_id, letter.title AS letter_title, letter.description AS letter_description, letter.from_id AS letter_from_id, letter.to_id AS letter_to_id, letter.datetime AS letter_datetime 
FROM letter
2016-11-12 22:00:21,926 INFO sqlalchemy.engine.base.Engine ()


[[ Letter: Hello World 
 From: john_doe (john@email.com)
 To: jane_doe (jane@email.com)
 Time: 2016-11-12 22:00:14.015000
 
 This is a test letter.
 HELLO WORLD!
 :)]]

In [14]:
session.query(LetterDetail).all()

2016-11-12 22:00:32,325 INFO sqlalchemy.engine.base.Engine SELECT letter.id AS letter_id, letter.title AS letter_title, letter.description AS letter_description, letter.from_id AS letter_from_id, letter.to_id AS letter_to_id, letter.datetime AS letter_datetime, from_user.id AS from_user_id, from_user.username AS from_user_username, from_user.email AS from_user_email, from_user.department_id AS from_user_department_id, to_user.id AS to_user_id, to_user.username AS to_user_username, to_user.email AS to_user_email, to_user.department_id AS to_user_department_id 
FROM letter JOIN users AS from_user ON from_user.id = letter.from_id JOIN users AS to_user ON to_user.id = letter.from_id
2016-11-12 22:00:32,342 INFO sqlalchemy.engine.base.Engine ()


In [15]:
session.query(LetterDetail).filter_by(letter_title='Hello World').all()

2016-11-12 22:00:34,036 INFO sqlalchemy.engine.base.Engine SELECT letter.id AS letter_id, letter.title AS letter_title, letter.description AS letter_description, letter.from_id AS letter_from_id, letter.to_id AS letter_to_id, letter.datetime AS letter_datetime, from_user.id AS from_user_id, from_user.username AS from_user_username, from_user.email AS from_user_email, from_user.department_id AS from_user_department_id, to_user.id AS to_user_id, to_user.username AS to_user_username, to_user.email AS to_user_email, to_user.department_id AS to_user_department_id 
FROM letter JOIN users AS from_user ON from_user.id = letter.from_id JOIN users AS to_user ON to_user.id = letter.from_id 
WHERE letter.title = ?
2016-11-12 22:00:34,145 INFO sqlalchemy.engine.base.Engine ('Hello World',)


In [16]:
session.query(LetterDetail).filter_by(from_user_email='john@email.com').all()

2016-11-12 22:00:35,753 INFO sqlalchemy.engine.base.Engine SELECT letter.id AS letter_id, letter.title AS letter_title, letter.description AS letter_description, letter.from_id AS letter_from_id, letter.to_id AS letter_to_id, letter.datetime AS letter_datetime, from_user.id AS from_user_id, from_user.username AS from_user_username, from_user.email AS from_user_email, from_user.department_id AS from_user_department_id, to_user.id AS to_user_id, to_user.username AS to_user_username, to_user.email AS to_user_email, to_user.department_id AS to_user_department_id 
FROM letter JOIN users AS from_user ON from_user.id = letter.from_id JOIN users AS to_user ON to_user.id = letter.from_id 
WHERE from_user.email = ?
2016-11-12 22:00:35,811 INFO sqlalchemy.engine.base.Engine ('john@email.com',)


In [17]:
session.close()

2016-11-12 22:00:38,618 INFO sqlalchemy.engine.base.Engine ROLLBACK
